# Formating our Data in X's and Y's
Before starting our modeling process, we need to format our data to read in the records in an X and Y format, where each X is a row, and each Y is a target variable.

In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("../Chicago-ETL-Processes/Data/crime_with_temp.csv")

In [4]:
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Reformatted Date,AVG TEMP
0,24170,JB429040,09/09/2018 10:30:00 PM,019XX E 74TH ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,False,False,...,01A,1190271.0,1856285.0,2018,09/16/2018 04:05:04 PM,41.760675,-87.578207,"(41.760674759, -87.578206773)",2018-09-09,69.111111
1,11445354,JB434245,09/09/2018 10:43:00 AM,034XX N ALBANY AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,08B,1155112.0,1922612.0,2018,09/16/2018 04:05:04 PM,41.943458,-87.705289,"(41.943458302, -87.705288503)",2018-09-09,69.111111
2,11445290,JB434151,09/09/2018 01:00:00 PM,018XX S KILDARE AVE,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,RESIDENCE,False,False,...,11,1147918.0,1890726.0,2018,09/16/2018 04:05:04 PM,41.856101,-87.732551,"(41.856101365, -87.73255109)",2018-09-09,69.111111
3,11445236,JB434294,09/09/2018 03:25:00 PM,025XX N ELSTON AVE,0560,ASSAULT,SIMPLE,SMALL RETAIL STORE,False,False,...,08A,1161467.0,1917112.0,2018,09/16/2018 04:05:04 PM,41.928236,-87.682084,"(41.928235818, -87.682084152)",2018-09-09,69.111111
4,11445151,JB433896,09/09/2018 09:30:00 AM,035XX N JANSSEN AVE,0820,THEFT,$500 AND UNDER,RESIDENTIAL YARD (FRONT/BACK),False,False,...,06,1166012.0,1923579.0,2018,09/16/2018 04:05:04 PM,41.945886,-87.665198,"(41.945885657, -87.66519775)",2018-09-09,69.111111


In [5]:
data.shape

(181816, 24)

Great! So we have over 180,000 records against 24 columns. In this isolated scenario we are going to use both the weather and previously criminal instances to predict the type, location, and level of severity of a criminal activity. Let's separate our x's and y's.

The y's we are most interested in predicting are as follows:

- Kidnapping
- Sexual assault 
- Homocide
- Motor vehicle theft 
- Weapons violation
- Battery
- Theft

Let's make a dictionary that associates each of these with a number.

In [30]:
types_of_crime = ["KIDNAPPING ", "SEX OFFENSE", "HOMICIDE", "MOTOR VEHICLE THEFT", "WEAPONS VIOLATION", "BATTERY", "THEFT"]

def get_crime_dict(crime_list):
    rt = {}
    for idx, crime in enumerate(crime_list):
        rt[crime] = idx
    return rt

crime_dict = get_crime_dict(types_of_crime)

In [19]:
def limit_by_crime_type(crime_list, df):
    return df[ df["Primary Type"].isin(crime_list)]
    
df = limit_by_crime_type(types_of_crime, data)

In [24]:
df.shape

(90133, 24)

In [25]:
df["Primary Type"].value_counts()

THEFT                  43542
BATTERY                34869
MOTOR VEHICLE THEFT     6808
WEAPONS VIOLATION       3814
SEX OFFENSE              704
HOMICIDE                 396
Name: Primary Type, dtype: int64

For the most basic version of the model, we'll just look at the temperature of the previous day. Once we have that training on SageMaker, we'll look at continuous integration pipelines to push our changes a LOT easier, then we'll iteratively improve the modeling strategy step by step. We can also design our system to send us updates when the model is producing low confidence scores, so that we can improve it as more data results come in.

In [40]:
def replace_crime_type_with_id(df, crime_dict):
    
    df["Target"] = [0 for i in range(df.shape[0])]
    
    for idx, row in df.iterrows():
        c = row["Primary Type"]
        c_id = crime_dict[c]
        df.at[idx, "Target"] = c_id        
    return df
    
replaced_df = replace_crime_type_with_id(df, crime_dict)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Ok, we have the target variable organized into a multi-class classification, now we need to formulate our x dependent variables. At this point it's helpful to draw a picture of your data, so you can mentally visualize how to organize the rows and columns. I've already done this on paper, so will continue here.

In [45]:
list(replaced_df)

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location',
 'Reformatted Date',
 'AVG TEMP',
 'Target']

In [63]:
xs_list = ["Target", "AVG TEMP", "Date", "Primary Type", "Block"]

model_df = replaced_df[xs_list]

This next step is perhaps the most simple approach to transforming the data, we're exploding out the categorical variables to create new columns for each unique value. This is useful so that we can feed each of our rows into different types of models, rather than being limited by only the ones that accept categorical values.

In [64]:
model_df = pd.get_dummies(model_df, columns = ["Block"])

Now that we have a loose format, let's store it on S3 so it lives past this notebook instance.

In [67]:
model_df.shape

(90133, 21041)

In [1]:
def limit_blocks(model_df):
    print (model_df.shape[0])
    
limit_blocks(model_df)

NameError: name 'model_df' is not defined

# Let's combine all of these steps into one main function with a few nicely refactored functions.

In [3]:
import pandas as pd

def get_crime_dict(crime_list):
    rt = {}
    for idx, crime in enumerate(crime_list):
        rt[crime] = idx
    return rt

def limit_by_crime_type(crime_list, df):
    return df[ df["Primary Type"].isin(crime_list)]


def replace_crime_type_with_id(df, crime_dict):
    
    df["Target"] = [0 for i in range(df.shape[0])]
    
    for idx, row in df.iterrows():
        c = row["Primary Type"]
        c_id = crime_dict[c]
        df.at[idx, "Target"] = c_id        
    return df


def main(f_name):
    data = pd.read_csv(f_name)
    
    types_of_crime = ["KIDNAPPING ", "SEX OFFENSE", "HOMICIDE", "MOTOR VEHICLE THEFT", "WEAPONS VIOLATION", "BATTERY", "THEFT"]

    crime_dict = get_crime_dict(types_of_crime)
    
    df = limit_by_crime_type(types_of_crime, data)
    
    replaced_df = replace_crime_type_with_id(df, crime_dict)
    
    xs_list = ["Target", "AVG TEMP", "Date", "Primary Type", "Block"]

    model_df = replaced_df[xs_list]
    
    return model_df
    
    
df = main("../Chicago-ETL-Processes/Data/crime_with_temp.csv")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


We started off with over 20,000 different blocks to classify against our 6 crime types, and that is too many for only 1 year of data for the purposes of demonstration. We'll limit that down to blocks that had over 10 counts of criminal activity in 2018.

In [19]:
def limit_blocks_by_count(df):

    blocks = {}

    for idx, row in df.iterrows():
        b = row["Block"]
        if b not in blocks:
            blocks[b] = 1
        else:
            blocks[b] += 1
            
    keepers = {}

    for k, v in blocks.items():
        if v > 10:
            keepers[k] = v
    
    return keepers

b = limit_blocks_by_count(df)

In [22]:
blocks = list(b.keys())

limited_by_blocks = df[ df["Block"].isin(blocks)]

In [23]:
limited_by_blocks.shape

(33063, 5)

In [25]:
limited_by_blocks["Target"].value_counts()

6    19635
5    10886
3     1344
4      890
1      213
2       95
Name: Target, dtype: int64

In [29]:
limited_by_blocks.head()

,Target,AVG TEMP,Date,Primary Type,Block
6,5,69.111111,09/09/2018 02:20:00 PM,BATTERY,057XX S CICERO AVE
15,6,69.111111,09/09/2018 12:00:00 PM,THEFT,015XX W DIVISION ST
20,5,69.111111,09/09/2018 04:20:00 PM,BATTERY,009XX W BELMONT AVE
25,6,69.111111,09/09/2018 04:00:00 PM,THEFT,025XX W CERMAK RD
30,6,69.111111,09/09/2018 08:00:00 AM,THEFT,013XX W BELMONT AVE


In [30]:
binary = pd.get_dummies(limited_by_blocks, columns=["Block"])

In [32]:
binary.shape

(33063, 1512)

In [34]:
binary.to_csv("Data/fewer_labeled_rows_by_block.csv", index=False)